In [105]:
from typing import List, Tuple
from typing import Optional
import math

import torch
import torch.nn as nn

import unittest
import numpy as np

from time import time

In [19]:
def create_binary_list_from_int(number: int) -> List[int]:
    if number < 0 or type(number) is not int:
        raise ValueError("Only Positive integers are allowed")

    return [int(x) for x in list(bin(number))[2:]]

In [20]:
def generate_even_data(max_int: int, batch_size: int=16)-> Tuple[List[int], List[List[int]]]:
    # Get the number of binary places needed to represent the maximum number
    max_length = int(math.log(max_int, 2))

    # Sample batch_size number of integers in range 0-max_int
    sampled_integers = np.random.randint(0, int(max_int / 2), batch_size)

    # create a list of labels all ones because all numbers are even
    labels = [1] * batch_size

    # Generate a list of binary numbers for training.
    data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]
    data = [([0] * (max_length - len(x))) + x for x in data]

    return labels, data

In [21]:
class Generator(nn.Module):

    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense_layer(x))

In [22]:
class Discriminator(nn.Module):
    def __init__(self, input_length: int):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1);
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))


In [48]:
import tenseal as ts
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
          )
context.generate_galois_keys()
context.global_scale = 2**40


In [337]:
# parameters
poly_mod_degree = 4096
coeff_mod_bit_sizes = [40, 20, 40]
# create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
# scale of ciphertext to use
ctx_eval.global_scale = 2 ** 20
# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()


# parameters
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
# create TenSEALContext
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

In [381]:
def train(max_int: int = 128, batch_size: int = 16, training_steps: int = 500):
    input_length = int(math.log(max_int, 2))

    # Models
    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    # Optimizers
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)

    # loss
    loss = nn.BCELoss()

    for i in range(training_steps):
        # zero the gradients on each iteration
        generator_optimizer.zero_grad()

        # Create noisy input for generator
        # Need float type instead of int
        noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
        generated_data = generator(noise)

        # Generate examples of even real data
        true_labels, true_data = generate_even_data(max_int, batch_size=batch_size)
        true_labels = torch.tensor(true_labels).float()
        true_labels = true_labels.unsqueeze(1)
        true_data = torch.tensor(true_data).float()
        
        # Train the generator
        # We invert the labels here and don't train the discriminator because we want the generator
        # to make things the discriminator classifies as true.
        generator_discriminator_out = discriminator(generated_data)
        generator_loss = loss(generator_discriminator_out, true_labels)
        generator_loss_vector = [generator_loss.item()]
        print(generator_loss_vector)
        enc_generator_loss = ts.ckks_vector(ctx_training, generator_loss_vector)
        print(enc_generator_loss)
        generator_loss.backward()
        generator_optimizer.step()

        # Train the discriminator on the true/generated data
        discriminator_optimizer.zero_grad()
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss(true_discriminator_out, true_labels)
        true_discriminator_loss_vector = [true_discriminator_loss.item()]
        print(true_discriminator_loss_vector)
        enc_true_discriminator_loss = ts.ckks_vector(ctx_training, true_discriminator_loss_vector)
        print(enc_true_discriminator_loss)

        # add .detach() here think about this
        generator_discriminator_out = discriminator(generated_data.detach())
        #generator_discriminator_loss = loss(generator_discriminator_out, torch.zeros(batch_size))
        generator_discriminator_loss = loss(generator_discriminator_out, torch.zeros(batch_size).unsqueeze(1))
        discriminator_loss = (true_discriminator_loss + generator_discriminator_loss) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()

In [382]:
train()

[0.6957530379295349]
[0.6755548715591431]
[0.6958860158920288]
[0.707151472568512]
[0.6996229887008667]
[0.7388465404510498]
[0.6939811706542969]
[0.7246197462081909]
[0.6993811726570129]
[0.6933154463768005]
[0.6982495188713074]
[0.6514685153961182]
[0.7014709115028381]
[0.7989068627357483]
[0.6960852146148682]
[0.6670610904693604]
[0.7028789520263672]
[0.7294723391532898]
[0.6859736442565918]
[0.7091749906539917]
[0.6957506537437439]
[0.7140580415725708]
[0.6973878145217896]
[0.7904215455055237]
[0.6919056177139282]
[0.6928750276565552]
[0.6938708424568176]
[0.7815135717391968]
[0.6866304278373718]
[0.7427546977996826]
[0.6881402134895325]
[0.7682679295539856]
[0.6958986520767212]
[0.6901689171791077]
[0.6932544708251953]
[0.7052214741706848]
[0.6840308904647827]
[0.710684061050415]
[0.6826352477073669]
[0.725522518157959]
[0.6886606812477112]
[0.7623162269592285]
[0.6860095262527466]
[0.7218835949897766]
[0.6883227229118347]
[0.7000504732131958]
[0.6858795881271362]
[0.6611105203628

[0.6694324016571045]
[0.7399413585662842]
[0.6692819595336914]
[0.791102409362793]
[0.675933301448822]
[0.7231051921844482]
[0.6706147789955139]
[0.8107243180274963]
[0.6684812307357788]
[0.6562548875808716]
[0.6682192087173462]
[0.7857681512832642]
[0.6622680425643921]
[0.7735699415206909]
[0.6721968650817871]
[0.7704461812973022]
[0.6703002452850342]
[0.6488312482833862]
[0.6662002801895142]
[0.6721445322036743]
[0.6671065092086792]
[0.7241411209106445]
[0.6666272878646851]
[0.7397778034210205]
[0.6639902591705322]
[0.7762866616249084]
[0.6628692746162415]
[0.6876960396766663]
[0.6643999218940735]
[0.6504216194152832]
[0.6738076210021973]
[0.6796453595161438]
[0.6726936101913452]
[0.6757684946060181]
[0.6678177118301392]
[0.7629067301750183]
[0.6645641326904297]
[0.6661536693572998]
[0.6657412648200989]
[0.667275071144104]
[0.6714890599250793]
[0.7561686038970947]
[0.6709896326065063]
[0.7952286005020142]
[0.6585415005683899]
[0.7388015389442444]
[0.6646364331245422]
[0.7221282720565

[0.7017472386360168]
[0.6682980060577393]
[0.7375670671463013]
[0.6635792255401611]
[0.7131248712539673]
[0.669019877910614]
[0.7232786417007446]
[0.6681989431381226]
[0.7206715941429138]
[0.6651473045349121]
[0.7696061730384827]
[0.6729294657707214]
[0.7665542960166931]
[0.6653000116348267]
[0.6555728316307068]
[0.6694227457046509]
[0.6905720233917236]
[0.6679146885871887]
[0.7492203116416931]
[0.6674996614456177]
[0.7401257157325745]
[0.665488064289093]
[0.7041752934455872]
[0.6545320749282837]
[0.7384882569313049]
[0.668396532535553]
[0.7507662773132324]
[0.6612675189971924]
[0.7647112607955933]
[0.6766625046730042]
[0.6770010590553284]
[0.6769852042198181]
[0.7318504452705383]
[0.6653479337692261]
[0.7332030534744263]
[0.6629634499549866]
[0.7762560844421387]
[0.6667540073394775]
[0.700777530670166]
[0.6581473350524902]
[0.7162764072418213]
[0.6703013777732849]
[0.7302020788192749]
[0.6620140671730042]
[0.7629009485244751]
[0.6703035235404968]
[0.7566134929656982]
[0.67841172218322

[0.7135911583900452]
[0.670712411403656]
[0.7324883937835693]
[0.6806483864784241]
[0.684090256690979]
[0.6718769073486328]
[0.7575572729110718]
[0.6702632904052734]
[0.7328557372093201]
[0.6656960248947144]
[0.7033036351203918]
[0.6678652763366699]
[0.6871503591537476]
[0.6670995950698853]
[0.7651565074920654]
[0.6865441203117371]
[0.720119833946228]
[0.6785286664962769]
[0.7448717355728149]
[0.6828422546386719]
[0.7144696116447449]
[0.6823749542236328]
[0.7410715222358704]
[0.6811048984527588]
[0.6557801961898804]
[0.6794703602790833]
[0.7230434417724609]
[0.6757970452308655]
[0.7140263319015503]
[0.6750601530075073]
[0.6851244568824768]
[0.6817034482955933]
[0.762245237827301]
[0.6720059514045715]
[0.7489698529243469]
[0.6763958930969238]
[0.7626124024391174]
[0.6803068518638611]
[0.7300382852554321]
[0.686081051826477]
[0.7051831483840942]
[0.671945333480835]
[0.7257263660430908]
[0.6778340339660645]
[0.7355940937995911]
[0.6768016815185547]
[0.708971381187439]
[0.6707887649536133]

[0.6865097880363464]
[0.7361537218093872]
[0.6790741682052612]
[0.7248713374137878]
[0.6808481812477112]
[0.7567507028579712]
[0.6951119899749756]
[0.7560464143753052]
[0.6819345951080322]
[0.7299984693527222]
[0.6880450248718262]
[0.7129800915718079]
[0.6884215474128723]
[0.7449675798416138]
[0.6892688870429993]
[0.746293842792511]
[0.6883156895637512]
[0.6985754370689392]
[0.6821098923683167]
[0.7219532132148743]
[0.6891427636146545]
[0.7472264170646667]
[0.6899267435073853]
[0.7174245715141296]
[0.7000117301940918]
[0.7235612273216248]
[0.681848406791687]
[0.7104501724243164]
[0.692738950252533]
[0.6951654553413391]
[0.6952576041221619]
[0.7254852652549744]
[0.6838989853858948]
[0.7505156993865967]
[0.6928824782371521]
[0.7149230241775513]
[0.6969884634017944]
[0.7103416919708252]
[0.6890500783920288]
[0.7080182433128357]
[0.6836445331573486]
[0.7464343309402466]
[0.6893748044967651]
[0.689767599105835]
[0.6965374946594238]
[0.6958793997764587]
[0.6949613094329834]
[0.73146098852157

[0.7169535160064697]
[0.7365039587020874]
[0.7085827589035034]
[0.7203186750411987]
[0.7040977478027344]
[0.718166172504425]
[0.7048399448394775]
[0.7084414958953857]
[0.7029199004173279]
[0.7340415716171265]
[0.7046116590499878]
[0.7140969634056091]
[0.7064101696014404]
[0.731026291847229]
[0.70960533618927]
[0.7385824918746948]
[0.7031320929527283]
[0.7427009344100952]
[0.7113603353500366]
[0.7063435316085815]
[0.7061924934387207]
[0.7170546650886536]
[0.7105172872543335]
[0.7273848652839661]
[0.7102148532867432]
[0.7029045224189758]
[0.706676721572876]
[0.7177134156227112]
[0.7143663167953491]
[0.7044724822044373]
[0.7048515677452087]
[0.715114951133728]
[0.7060103416442871]
[0.7277612090110779]
[0.7082133889198303]
[0.7034022808074951]
[0.7188243865966797]
[0.7156106233596802]
[0.7134525775909424]
[0.7195090055465698]
[0.7074129581451416]
[0.6974419951438904]
[0.7175149917602539]
[0.7145771980285645]
[0.7098501324653625]
[0.7208482623100281]
[0.7120741009712219]
[0.7248975038528442

[0.7079766392707825]
[0.7288232445716858]
[0.6871038675308228]
[0.7281314134597778]
[0.7050472497940063]
[0.7240503430366516]
[0.7077516317367554]
[0.731705904006958]
[0.7029246687889099]
[0.7320269346237183]
[0.7173206210136414]
[0.7313838005065918]
[0.7183603048324585]
[0.732558012008667]
[0.703953742980957]
[0.7319697141647339]
[0.6997531056404114]
[0.7327643632888794]
[0.7061074376106262]
[0.7352755665779114]
[0.7030705213546753]
[0.7379963994026184]
[0.705278217792511]
[0.7345598936080933]
[0.7101787328720093]
[0.7386856079101562]
[0.7026768922805786]
[0.7394691705703735]
[0.7054805755615234]
[0.7330918312072754]
[0.7142686247825623]
[0.7376464605331421]
[0.6979382038116455]
[0.7360882759094238]
[0.7011825442314148]
[0.738263726234436]
[0.6896446943283081]
[0.7376260161399841]
[0.7089514136314392]
[0.7335606217384338]
[0.6909615993499756]
[0.7328075766563416]
[0.7128760814666748]
[0.736822783946991]
[0.6837282776832581]
[0.735364556312561]
[0.7151788473129272]
[0.7351638078689575]

[0.6868869066238403]
[0.7442229986190796]
[0.7022984027862549]
[0.741701602935791]
[0.697770893573761]
[0.7452989816665649]
[0.6988235116004944]
[0.7438440918922424]
[0.6679560542106628]
[0.744310200214386]
[0.7005505561828613]
[0.7460139989852905]
[0.7159196734428406]
[0.7429702877998352]
[0.6747297048568726]
[0.7418062686920166]
[0.6939061284065247]
[0.7426934242248535]
[0.708743691444397]
[0.7430709004402161]
[0.6723973751068115]
[0.7464513778686523]
[0.7122690081596375]
[0.7470216155052185]
[0.6844109892845154]
[0.741452693939209]
[0.6757417917251587]
[0.7441247701644897]
[0.6815690398216248]
[0.7427996397018433]
[0.6718829274177551]
[0.7407426238059998]
[0.6867453455924988]
[0.7461947202682495]
[0.6752243638038635]
[0.7431333065032959]
[0.690970242023468]
[0.7436705231666565]
[0.6807584166526794]
[0.7441644072532654]
[0.6908624172210693]
[0.7485164403915405]
[0.678368091583252]
[0.7443771362304688]
[0.6916778087615967]
[0.7447472810745239]
[0.692022442817688]
[0.746330738067627]
[

[0.6999778151512146]
[0.747442364692688]
[0.6783338785171509]
[0.7446959018707275]
[0.6618216037750244]
[0.7386131882667542]
[0.6828246712684631]
[0.7444108128547668]
[0.6830739378929138]
[0.745570182800293]
[0.6734238862991333]
[0.741466760635376]
[0.6723717451095581]
[0.7415813207626343]
[0.686323881149292]
[0.7395848631858826]
[0.687865138053894]
[0.7442963719367981]
[0.6720829010009766]
[0.7417635917663574]
[0.687279999256134]
[0.7450213432312012]
[0.6791936755180359]
[0.7416843771934509]
[0.6879634857177734]
[0.7457564473152161]
[0.7182939648628235]
[0.7446527481079102]
[0.6597347855567932]


In [264]:
y_train, x_train = generate_even_data(max_int=128, batch_size=16)
print(y_train)
y_train
print(x_train)
x_test = [115, 90, 94, 122, 122, 122, 95, 115, 83, 123, 114, 123, 94, 89, 91, 86]
y_test = [0,1,1,1,1,1,0,0,0,0,1,0,1,0,0,1]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[1, 1, 0, 0, 0, 1, 0], [1, 0, 1, 0, 1, 1, 0], [1, 1, 0, 0, 1, 1, 0], [1, 1, 0, 0, 1, 0, 0], [0, 1, 1, 1, 1, 0, 0], [0, 1, 0, 1, 1, 1, 0], [1, 0, 1, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 0], [0, 0, 1, 0, 0, 1, 0], [1, 1, 0, 1, 1, 1, 0], [0, 0, 1, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0]]


In [265]:
class LR(torch.nn.Module):

    def __init__(self, n_features):
        super(LR, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out

In [266]:
#n_features = x_train.shape[1]
n_features = 7
model = LR(n_features)
# use gradient descent with a learning_rate=1
optim = torch.optim.SGD(model.parameters(), lr=1)
# use Binary Cross Entropy Loss
criterion = torch.nn.BCELoss()

In [267]:
class EncryptedLR:
    
    def __init__(self, torch_lr):
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()
        # we accumulate gradients and counts the number of iterations
        self._delta_w = 0
        self._delta_b = 0
        self._count = 0
        
    def forward(self, enc_x):
        enc_out = enc_x.dot(self.weight) + self.bias
        enc_out = EncryptedLR.sigmoid(enc_out)
        return enc_out
    
    def backward(self, enc_x, enc_out, enc_y):
        out_minus_y = (enc_out - enc_y)
        self._delta_w += enc_x * out_minus_y
        self._delta_b += out_minus_y
        self._count += 1
        
    def update_parameters(self):
        if self._count == 0:
            raise RuntimeError("You should at least run one forward iteration")
        # update weights
        # We use a small regularization term to keep the output
        # of the linear layer in the range of the sigmoid approximation
        self.weight -= self._delta_w * (1 / self._count) + self.weight * 0.05
        self.bias -= self._delta_b * (1 / self._count)
        # reset gradient accumulators and iterations count
        self._delta_w = 0
        self._delta_b = 0
        self._count = 0
    
    @staticmethod
    def sigmoid(enc_x):
        # We use the polynomial approximation of degree 3
        # sigmoid(x) = 0.5 + 0.197 * x - 0.004 * x^3
        # from https://eprint.iacr.org/2018/462.pdf
        # which fits the function pretty well in the range [-5,5]
        return enc_x.polyval([0.5, 0.197, 0, -0.004])
    
    def plain_accuracy(self, x_test, y_test):
        # evaluate accuracy of the model on
        # the plain (x_test, y_test) dataset
        w = torch.tensor(self.weight)
        b = torch.tensor(self.bias)
        out = torch.sigmoid(x_test.matmul(w) + b).reshape(-1, 1)
        correct = torch.abs(y_test - out) < 0.5
        return correct.float().mean()    
    
    def encrypt(self, context):
        self.weight = ts.ckks_vector(context, self.weight)
        self.bias = ts.ckks_vector(context, self.bias)
        
    def decrypt(self):
        self.weight = self.weight.decrypt()
        self.bias = self.bias.decrypt()
        
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [268]:
# parameters
poly_mod_degree = 4096
coeff_mod_bit_sizes = [40, 20, 40]
# create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
# scale of ciphertext to use
ctx_eval.global_scale = 2 ** 20
# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()


# parameters
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
# create TenSEALContext
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

In [269]:
t_start = time()
#enc_x_train = [ts.ckks_vector(ctx_training, x.tolist()) for x in x_train]
#enc_y_train = [ts.ckks_vector(ctx_training, y.tolist()) for y in y_train]
enc_x_train_array = []
enc_y_train_array = []
for x,y in zip(x_train,y_train):
    enc_x_train = ts.ckks_vector(ctx_training, x)
    enc_x_train_array.append(enc_x_train)
    #print("Encryption of ", x," is ", enc_x_train)
    
    # convert single digit to vector
    y = [int(d) for d in str(y)]
    enc_y_train = ts.ckks_vector(ctx_training, y)
    enc_y_train_array.append(enc_y_train)
    #print("Encryption of ", y," is ", enc_y_train)

    
    
#enc_x_train = [ts.ckks_vector(ctx_training, x_train) for x in x_train]
#enc_y_train = [ts.ckks_vector(ctx_training, y_train) for y in y_train]
                                 
t_end = time()
print(f"Encryption of the training_set took {int(t_end - t_start)} seconds")

Encryption of the training_set took 0 seconds


In [270]:
eelr = EncryptedLR(LR(n_features))
#accuracy = eelr.plain_accuracy(x_test, y_test)
#print(f"Accuracy at epoch #0 is {accuracy}")
EPOCHS = 5

times = []
for epoch in range(EPOCHS):
    eelr.encrypt(ctx_training)
    
    # if you want to keep an eye on the distribution to make sure
    # the function approxiamation is still working fine
    # WARNING: this operation is time consuming
    # encrypted_out_distribution(eelr, enc_x_train)
    
    t_start = time()
    for enc_x, enc_y in zip(enc_x_train_array, enc_y_train_array):
        enc_out = eelr.forward(enc_x)
        eelr.backward(enc_x, enc_out, enc_y)
     
    eelr.update_parameters()
    t_end = time()
    times.append(t_end - t_start)
    
    eelr.decrypt()
    #accuracy = eelr.plain_accuracy(x_test, y_test)
    #print(f"Accuracy at epoch #{epoch + 1} is {accuracy}")


print(f"\nAverage time per epoch: {int(sum(times) / len(times))} seconds")
#print(f"Final accuracy is {accuracy}")

#diff_accuracy = plain_accuracy - accuracy
#print(f"Difference between plain and encrypted accuracies: {diff_accuracy}")
#if diff_accuracy < 0:
#    print("Oh! We got a better accuracy when training on encrypted data! The noise was on our side...")


Average time per epoch: 1 seconds
